# Project -1 : Counting Cells by finding area of higher tensity color 

### Installing required packages

In [1]:
#!/usr/local/opt/python/bin/python2.7 -m pip install Pillow

In [2]:
#!pip2 install tqdm

In [68]:
#!pip3 install scikit-image

In [20]:
#!pip3 install scikit-learn

  Using cached scikit_learn-0.19.1-cp36-cp36m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl


In [22]:
#!pip3 install scipy

  Using cached scipy-1.0.0-cp36-cp36m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl


In [19]:
#import sys
#print(sys.executable)

/usr/local/opt/python3/bin/python3.6


In [17]:
#!brew install openslide
#!/usr/local/opt/python/bin/python2.7 -m pip install openslide-python

## Configuration file

In [4]:
import os

# directory of the config file
CURRENT_DIR = os.path.realpath('.')

DEFAULT_FILENAME = 'slide'

IMAGE_FOLDER_PATH = CURRENT_DIR+"/input"

OUTPUT_FOLDER_PATH = CURRENT_DIR+"/output"

IMG_FORMAT = 'jpeg'

TILE_SIZE = 1000

OVERLAP = 0

LIMIT_BOUNDS = True

QUALITY = 100

NUM_WORKERS = 12

ONLY_LAST = True

SAVE_REJECTED = False

DONT_REJECT = True

# increase this to reject more
REJECT_THRESHOLD = 200

ROTATE = False

MAX_WHITE_SIZE = (TILE_SIZE*TILE_SIZE)/2

def ver_print(string, value):
    print(string + " {0}".format(value))

## Preprocessing and tilling

In [5]:
import glob
import os
from tqdm import tqdm
import PIL
import cv2
from PIL import Image
from tiler import WholeSlideTiler

def main():
    files = glob.glob(IMAGE_FOLDER_PATH + '/*.svs')
    for slidepath in tqdm(files):
        basename = os.path.splitext(os.path.basename(slidepath))[0]
        basepath = os.path.join(OUTPUT_FOLDER_PATH, basename)
        WholeSlideTiler(slidepath, basepath, IMG_FORMAT, TILE_SIZE, OVERLAP, LIMIT_BOUNDS, ROTATE,
                        QUALITY, NUM_WORKERS, ONLY_LAST).run()


if __name__ == '__main__':
    main()


Tiling slide: wrote 5796/5796 tiless]


Tiling completed on slide in:  <time here>
Tiling completed on label in:  <time here>


Tiling label: wrote 1/1 tiles
Tiling macro: wrote 2/2 tiles
Tiling thumbnail: wrote 2/2 tiles
100%|██████████| 1/1 [05:41<00:00, 341.79s/it]

Tiling completed on macro in:  <time here>
Tiling completed on thumbnail in:  <time here>


## Normalizing

In [2]:
from __future__ import division

import glob

import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image 
from numpy import random
import os


def safe_ln(x, minval=0.0000000001):
    return np.log(x.clip(min=minval))


def normalize_staining(I=None):
    I = I.astype(np.float64)
    Io = 240
    beta = 0.15
    alpha = 1
    HERef = np.array([[0.5626, 0.2159], [0.7201, 0.8012], [0.4062, 0.5581]])
    maxCRef = np.array([1.9705, 1.0308])

    (h, w, c) = np.shape(I)
    I = np.reshape(I, (h * w, c), order='F')

    # Step 1. Convert RGB to OD.
    OD = - np.log((I + 1) / Io) # optical density where each channel in the image is normalized to values between [0, 1]

    # Step 2. Remove data with OD intensity less than beta
    ODhat = (OD[(np.logical_not((OD < beta).any(axis=1))), :])

    # Step 3. Calculate SVD on the OD tuples
    cov = np.cov(ODhat, rowvar=False)
    (W, V) = np.linalg.eig(cov)

    # Step 4. create plane from the SVD directions
    # corresponding to the two largest singular values
    Vec = - np.transpose(np.array([V[:, 1], V[:, 0]]))

    # Step 5. Project data onto the plane and normalize to unit Length
    That = np.dot(ODhat, Vec)

    # Step 6. Calculate angle of each point w.r.t the first SVD direction
    phi = np.arctan2(That[:, 1], That[:, 0])

    # Step 7. Find robust extremes (some alpha th and (100 - alpha th) percentiles of the angle
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100 - alpha)
    vMin = np.dot(Vec, np.array([np.cos(minPhi), np.sin(minPhi)]))
    vMax = np.dot(Vec, np.array([np.cos(maxPhi), np.sin(maxPhi)]))
    if vMin[0] > vMax[0]:
        HE = np.array([vMin, vMax])
    else:
        HE = np.array([vMax, vMin])

    HE = np.transpose(HE)

    # Step 8. Convert extreme values back to OD space
    Y = np.transpose(np.reshape(OD, (h * w, c)))
    C = np.linalg.lstsq(HE, Y,rcond=None)
    maxC = np.percentile(C[0], 99, axis=1)
    C = C[0] / maxC[:, None]
    C = C * maxCRef[:, None]
    Inorm = Io * np.exp(- np.dot(HERef, C))
    Inorm = np.reshape(np.transpose(Inorm), (h, w, c), order='F')
    Inorm = np.clip(Inorm, 0, 255)
    Inorm = np.array(Inorm, dtype=np.uint8)

    return Inorm  # ,H,E


## Selecting random 10 tiles for Normalization

In [3]:
def main():

    list_img = glob.glob(glob.glob("output/*/")[0]+"slide/18/*.jpeg")
    for filename in random.choice(list_img,10):
        fileout = os.path.splitext(filename)[0]+"_normal.jpeg"
        img = cv2.imread(filename)
        img = img.astype(np.float64)

        img = normalize_staining(img)

        cv2.imwrite(fileout, img)
        img_show = Image.open(fileout)
        img_show.show()  
        
main()


# Finding the main colors in tiles

## Loading and Resizing tiles

In [10]:
# Load the image
from skimage import io, color
import cv2
import numpy as np

image = cv2.imread("19_15_1.jpeg")
#convert color space from RGB to LAB
image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
 
# Resize it
h, w, _ = image.shape
w_new = int(100 * w / max(w, h) )
h_new = int(100 * h / max(w, h) )
 
image = cv2.resize(image, (w_new, h_new));

# This line is to set L values to 0 and keep A and B
image[:,:,0]= 0
#print(image)


#cv2.imshow('image',image)
#cv2.waitKey(0)
#print(image[0][0])


## Step 1: Clustering colours with K-Means

In [11]:
from sklearn.cluster import KMeans
 
# Reshape the image to be a list of pixels
image_array = image.reshape((image.shape[0] * image.shape[1], 3))
 
# Clusters the pixels
clt = KMeans(n_clusters = 3)
clt.fit(image_array)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

## Step 2: Sort clusters by size

In [12]:
# By Adrian Rosebrock
import numpy as np
import cv2
 
def centroid_histogram(clt):
 # grab the number of different clusters and create a histogram
 # based on the number of pixels assigned to each cluster
 numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
 (hist, _) = np.histogram(clt.labels_, bins = numLabels)
 
 # normalize the histogram, such that it sums to one
 hist = hist.astype("float")
 hist /= hist.sum()
 
 # return the histogram
 return hist

print(clt.fit)
print(clt.cluster_centers_)
# Finds how many pixels are in each cluster
hist = centroid_histogram(clt)

print(hist)
 
# Sort the clusters according to how many pixel they have
zipped = zip (hist, clt.cluster_centers_)
hist, clt.cluster_centers = zip(*zipped)
   
 


<bound method KMeans.fit of KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)>
[[  0.         131.48542459 126.44011407]
 [  0.         161.90214425  91.36179337]
 [  0.         152.10119187 115.87660668]]
[0.3156 0.2565 0.4279]


## Step 3: Clustering evaluation

In [13]:
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score

bestSilhouette = -1
bestClusters = 0;

for clusters in range(2, 6):
	# Cluster colours
	clt = KMeans(n_clusters = clusters)
	clt.fit(image_array)

	# Validate clustering result
	silhouette = metrics.silhouette_score(image_array, clt.labels_, metric='euclidean')

	# Find the best one
	if silhouette > bestSilhouette:
		bestSilhouette = silhouette;
		bestClusters = clusters;
        
print(bestClusters)

2


## Counting colors

In [8]:
from PIL import Image , ImageStat


im = Image.open("112_16_1.jpeg").convert('L')
stat = ImageStat.Stat(im)
print "Read RMS brightness of image: "
print stat.extrema
print stat.count
print stat.sum


black = 0
red = 0

for pixel in im.getdata():
    if pixel == (0, 0, 0): # if your image is RGB (if RGBA, (0, 0, 0, 255) or so
        black += 1
    else:
        red += 1
print('black=' + str(black)+', red='+str(red))

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Read RMS brightness of image: ")? (<ipython-input-8-2d4c635cd774>, line 6)